# KNN Starter for Brain Comp

Kaggleの脳波コンペ用のStarter Notebookである。SpectrogramとEEGを使用（UPDATE済み）。より多くのSpectrogram / EEG特徴量をエンジニアリングすることで、CV / LBスコアを改善できる。

# Load Libraries

In [1]:
import os, gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

VER = 1

# Load Train Data

In [2]:
# df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
df = pd.read_csv('../input/train.csv')
TARGETS = df.columns[-6:]
print('Train shape:', df.shape )
print('Targets', list(TARGETS))
df.head()

Train shape: (106800, 15)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']


,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


# Feature Engineer
特徴量を作成する。

In [4]:
# Falseのとき、CHRIS DEOTTE氏が作成したデータを利用
READ_SPEC_FILES = False
READ_EEG_SPEC_FILES = False

In [5]:
%%time
# READ ALL SPECTROGRAMS
# スペクトログラムの読み込み
# PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_spectrograms/'
PATH = '../input/hms-harmful-brain-activity-classification/train_spectrograms/'
files = os.listdir(PATH)
print(f'There are {len(files)} spectrogram parquets')

if READ_SPEC_FILES:    
    spectrograms = {}
    for i,f in enumerate(files):
        if i%100==0: print(i,', ',end='')
        tmp = pd.read_parquet(f'{PATH}{f}')
        name = int(f.split('.')[0])
        spectrograms[name] = tmp.iloc[:,1:].values
else:
    # spectrograms = np.load('/kaggle/input/brain-spectrograms/specs.npy',allow_pickle=True).item()
    spectrograms = np.load('../input/hms-harmful-brain-activity-classification/brain-spectrograms/specs.npy',allow_pickle=True).item()

There are 11138 spectrogram parquets
CPU times: user 3.36 s, sys: 6.29 s, total: 9.65 s
Wall time: 1min 27s


In [6]:
%%time
# READ ALL EEG SPECTROGRAMS
# 生のEEGデータから作成されたEEG スペクトログラムの読み込み
if READ_EEG_SPEC_FILES:
    all_eegs = {}
    for i,e in enumerate(train.eeg_id.values):
        if i%100==0: print(i,', ',end='')
        x = np.load(f'../input/hms-harmful-brain-activity-classification/brain-eeg-spectrograms/EEG_Spectrograms/{e}.npy')
        all_eegs[e] = x
else:
    # all_eegs = np.load('/kaggle/input/hms-harmful-brain-activity-classification/brain-eeg-spectrograms/eeg_specs.npy',allow_pickle=True).item()
    all_eegs = np.load('../input/hms-harmful-brain-activity-classification/brain-eeg-spectrograms/eeg_specs.npy',allow_pickle=True).item()

CPU times: user 5.56 s, sys: 8.52 s, total: 14.1 s
Wall time: 1min 46s


In [7]:
%time
# ENGINEER FEATURES

import warnings
warnings.filterwarnings('ignore')

# FEATURE NAMES
# 元の Spectrogram のファイルから、列名を取得
SPEC_COLS = pd.read_parquet(f'{PATH}1000086677.parquet').columns[1:]
FEATURES = [f'{c}_mean_10m' for c in SPEC_COLS]             # スペクトログラム: 10分の時間窓の時間平均
FEATURES += [f'{c}_min_10m' for c in SPEC_COLS]             # スペクトログラム: 10分の時間窓の時間最小値
FEATURES += [f'{c}_mean_20s' for c in SPEC_COLS]            # スペクトログラム: 20秒の時間窓の時間平均
FEATURES += [f'{c}_min_20s' for c in SPEC_COLS]             # スペクトログラム: 20秒の時間窓の時間最小値
FEATURES += [f'eeg_mean_f{x}_10s' for x in range(512)]      # 脳波: 10分の時間窓の時間平均
FEATURES += [f'eeg_min_f{x}_10s' for x in range(512)]       # 脳波: 10分の時間窓の時間最小値
FEATURES += [f'eeg_max_f{x}_10s' for x in range(512)]       # 脳波: 20秒の時間窓の時間平均
FEATURES += [f'eeg_std_f{x}_10s' for x in range(512)]       # 脳波: 20秒の時間窓の時間平均
# print(f'We are creating {len(FEATURES)} features for {len(train)} rows... ',end='')

# data = np.zeros((len(train),len(FEATURES)))
# for k in range(len(train)):
#     if k%100==0: print(k,', ',end='')
#     row = train.iloc[k]
#     r = int( (row['min'] + row['max'])//4 ) 

#     ### スペクトログラム特徴量の計算
#     # 10 MINUTE WINDOW FEATURES (MEANS and MINS)
#     # 10分時間窓の特徴量計算
#     x = np.nanmean(spectrograms[row.spec_id][r:r+300,:],axis=0)
#     data[k,:400] = x
#     x = np.nanmin(spectrograms[row.spec_id][r:r+300,:],axis=0)
#     data[k,400:800] = x

#     # 20 SECOND WINDOW FEATURES (MEANS and MINS)
#     # 20秒時間窓の特徴量計算
#     x = np.nanmean(spectrograms[row.spec_id][r+145:r+155,:],axis=0)
#     data[k,800:1200] = x
#     x = np.nanmin(spectrograms[row.spec_id][r+145:r+155,:],axis=0)
#     data[k,1200:1600] = x

#     ### EEG特徴量の計算
#     # RESHAPE EEG SPECTROGRAMS 128x256x4 => 512x256
#     eeg_spec = np.zeros((512,256),dtype='float32')
#     xx = all_eegs[row.eeg_id]
#     for j in range(4): eeg_spec[128*j:128*(j+1),] = xx[:,:,j]

#     # 10 SECOND WINDOW FROM EEG SPECTROGRAMS 
#     # 10秒時間窓の特徴量の計算（mean, min, max, std）
#     x = np.nanmean(eeg_spec.T[100:-100,:],axis=0)
#     data[k,1600:2112] = x
#     x = np.nanmin(eeg_spec.T[100:-100,:],axis=0)
#     data[k,2112:2624] = x
#     x = np.nanmax(eeg_spec.T[100:-100,:],axis=0)
#     data[k,2624:3136] = x
#     x = np.nanstd(eeg_spec.T[100:-100,:],axis=0)
#     data[k,3136:3648] = x

# train[FEATURES] = data
# print(); print('New train shape:',train.shape)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 6.44 µs
We are creating 3648 features for 17089 rows... 0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , 3900 , 4000 , 4100 , 4200 , 4300 , 4400 , 4500 , 4600 , 4700 , 4800 , 4900 , 5000 , 5100 , 5200 , 5300 , 5400 , 5500 , 5600 , 5700 , 5800 , 5900 , 6000 , 6100 , 6200 , 6300 , 6400 , 6500 , 6600 , 6700 , 6800 , 6900 , 7000 , 7100 , 7200 , 7300 , 7400 , 7500 , 7600 , 7700 , 7800 , 7900 , 8000 , 8100 , 8200 , 8300 , 8400 , 8500 , 8600 , 8700 , 8800 , 8900 , 9000 , 9100 , 9200 , 9300 , 9400 , 9500 , 9600 , 9700 , 9800 , 9900 , 10000 , 10100 , 10200 , 10300 , 10400 , 10500 , 10600 , 10700 , 10800 , 10900 , 11000 , 11100 , 11200 , 11300 , 11400 , 11500 , 11600 , 11700 , 11800 , 11900 , 12000 , 12100 , 12200 , 12300 , 12400 , 

In [8]:
# # FREE MEMORY
# del all_eegs, spectrograms, data
# gc.collect()

0

# Infer Test and Create Submission CSV
Below we use our 5 CatBoost fold models to infer the test data and create a `submission.csv` file.

5foldの各CatBoostモデルを用いてテストデータを予測

In [16]:
# del train; gc.collect()
# test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
test = pd.read_csv('../input/hms-harmful-brain-activity-classification/test.csv')
print('Test shape',test.shape)
test.head()

Test shape (1, 3)


,spectrogram_id,eeg_id,patient_id
0,853520,3911565283,6885


In [17]:
import pywt, librosa

USE_WAVELET = None 

NAMES = ['LL','LP','RP','RR']

FEATS = [['Fp1','F7','T3','T5','O1'],
         ['Fp1','F3','C3','P3','O1'],
         ['Fp2','F8','T4','T6','O2'],
         ['Fp2','F4','C4','P4','O2']]

# DENOISE FUNCTION
# dの平均からの絶対偏差の平均を返す
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

# wavelet関数を使って雑音のしきい値を計算し、それよりも小さい周波数成分を除去する
def denoise(x, wavelet='haar', level=1):    
    # xにウェーブレット変換(DWT)し、係数を計算する
    coeff = pywt.wavedec(x, wavelet, mode="per")
    # 係数の最後のレベル（詳細係数）からノイズの標準偏差を計算
    sigma = (1/0.6745) * maddest(coeff[-level])

    # ノイズのしきい値を計算
    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    # しきい値処理し、ノイズを除去
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:]) # 

    # 修正された係数を使用して逆ウェーブレット変換(IDWT)を施し、ノイズ除去された信号を返す
    ret=pywt.waverec(coeff, wavelet, mode='per') 
    
    return ret


# EEG信号からスペクトログラムを計算する関数
def spectrogram_from_eeg(parquet_path, display=False):
    # EEGデータは4つのモンタージュに分かれている。
    # 各モンタージュのスペクトログラムを計算して、4つのスペクトログラムを結合した3次元配列として返す
    
    # LOAD MIDDLE 50 SECONDS OF EEG SERIES
    eeg = pd.read_parquet(parquet_path)
    middle = (len(eeg)-10_000)//2
    eeg = eeg.iloc[middle:middle+10_000]
    
    # VARIABLE TO HOLD SPECTROGRAM
    img = np.zeros((128,256,4),dtype='float32')
    
    if display: plt.figure(figsize=(10,7))
    signals = []
    # EEG信号は4つのモンタージュに分かれているため、スペクトログラムを作成するモンタージュを選択
    for k in range(4):
        COLS = FEATS[k]
        
        # 差分をとるモンタージュを選択
        for kk in range(4):
        
            # COMPUTE PAIR DIFFERENCES: モンタージュ信号間の差分をとる
            x = eeg[COLS[kk]].values - eeg[COLS[kk+1]].values

            # FILL NANS
            m = np.nanmean(x)
            if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
            else: x[:] = 0

            # DENOISE
            if USE_WAVELET:
                x = denoise(x, wavelet=USE_WAVELET)
            signals.append(x)

            # RAW SPECTROGRAM: 差分信号に対してメル周波数スペクトログラムを計算する
            mel_spec = librosa.feature.melspectrogram(y=x, sr=200, hop_length=len(x)//256, 
                  n_fft=1024, n_mels=128, fmin=0, fmax=20, win_length=128)

            # LOG TRANSFORM: ログ変換
            width = (mel_spec.shape[1]//32)*32
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:,:width]

            # STANDARDIZE TO -1 TO 1: 標準化
            mel_spec_db = (mel_spec_db+40)/40 
            img[:,:,k] += mel_spec_db
                
        # AVERAGE THE 4 MONTAGE DIFFERENCES
        # 各モンタージュ間の差分の平均をとる
        img[:,:,k] /= 4.0
        
        if display:
            plt.subplot(2,2,k+1)
            plt.imshow(img[:,:,k],aspect='auto',origin='lower')
            plt.title(f'EEG {eeg_id} - Spectrogram {NAMES[k]}')
            
    if display: 
        plt.show()
        plt.figure(figsize=(10,5))
        offset = 0
        for k in range(4):
            if k>0: offset -= signals[3-k].min()
            plt.plot(range(10_000),signals[k]+offset,label=NAMES[3-k])
            offset += signals[3-k].max()
        plt.legend()
        plt.title(f'EEG {eeg_id} Signals')
        plt.show()
        print(); print('#'*25); print()
        
    return img

In [18]:
# CREATE ALL EEG SPECTROGRAMS
# テストデータのeegを用いてスペクトログラムを計算
# PATH2 = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/'
PATH2 = '../input/hms-harmful-brain-activity-classification/test_eegs/'
DISPLAY = 0
EEG_IDS2 = test.eeg_id.unique()
all_eegs2 = {}

print('Converting Test EEG to Spectrograms...'); print()
for i,eeg_id in enumerate(EEG_IDS2):
        
    # CREATE SPECTROGRAM FROM EEG PARQUET
    img = spectrogram_from_eeg(f'{PATH2}{eeg_id}.parquet', i<DISPLAY)
    all_eegs2[eeg_id] = img

Converting Test EEG to Spectrograms...



In [19]:
# FEATURE ENGINEER TEST
# trainデータと同じ特徴量の作成

# PATH2 = '/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/'
PATH2 = '../input/hms-harmful-brain-activity-classification/test_spectrograms/'
data = np.zeros((len(test),len(FEATURES)))
    
for k in range(len(test)):
    row = test.iloc[k]
    s = int( row.spectrogram_id )
    spec = pd.read_parquet(f'{PATH2}{s}.parquet')
    
    # 10 MINUTE WINDOW FEATURES
    x = np.nanmean( spec.iloc[:,1:].values, axis=0)
    data[k,:400] = x
    x = np.nanmin( spec.iloc[:,1:].values, axis=0)
    data[k,400:800] = x

    # 20 SECOND WINDOW FEATURES
    x = np.nanmean( spec.iloc[145:155,1:].values, axis=0)
    data[k,800:1200] = x
    x = np.nanmin( spec.iloc[145:155,1:].values, axis=0)
    data[k,1200:1600] = x
    
    # RESHAPE EEG SPECTROGRAMS 128x256x4 => 512x256
    eeg_spec = np.zeros((512,256),dtype='float32')
    xx = all_eegs2[row.eeg_id]
    for j in range(4): eeg_spec[128*j:128*(j+1),] = xx[:,:,j]

    # 10 SECOND WINDOW FROM EEG SPECTROGRAMS 
    x = np.nanmean(eeg_spec.T[100:-100,:],axis=0)
    data[k,1600:2112] = x
    x = np.nanmin(eeg_spec.T[100:-100,:],axis=0)
    data[k,2112:2624] = x
    x = np.nanmax(eeg_spec.T[100:-100,:],axis=0)
    data[k,2624:3136] = x
    x = np.nanstd(eeg_spec.T[100:-100,:],axis=0)
    data[k,3136:3648] = x

test[FEATURES] = data
print('New test shape',test.shape)

New test shape (1, 3651)


In [20]:
# INFER CATBOOST ON TEST
from joblib import dump, load
import catboost as cat
from catboost import CatBoostClassifier, Pool
preds = []

for i in range(5):
    print(i,', ',end='')
    model = CatBoostClassifier(task_type='GPU')
    # model.load_model(f'/kaggle/input/models/model/CAT_v{VER}_f{i}.cat')
    model.load(f'../hms-harmful-brain-activity-classification/knn-ver{VER}-train/models/KNN_v{VER}_f{i}.cat')


    pred = model.predict_proba(test)
    preds.append(pred)
pred = np.mean(preds,axis=0)
print()
print('Test preds shape',pred.shape)

0 , 1 , 2 , 3 , 4 , 
Test preds shape (1, 6)


In [21]:
sub = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub[TARGETS] = pred
sub.to_csv('submission.csv',index=False)
print('Submissionn shape',sub.shape)
sub.head()

Submissionn shape (1, 7)


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.343437,0.007858,0.000378,0.129815,0.11379,0.404722


In [22]:
# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
sub.iloc[:,-6:].sum(axis=1)

0    1.0
dtype: float64